In [1]:
import netCDF4 as nc
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.axes_grid1 import ImageGrid
from matplotlib.animation import FuncAnimation
from matplotlib.backends.backend_pdf import PdfPages

import os
import glob
import pandas as pd
import json
from dotenv import load_dotenv
from functools import partial

from utils.db_tools import get_db, classify_trajectories, metrics_grid, plot_grid, make_animation

In [2]:
model = "bruss"
run_id = "blowup_test"
load_dotenv()
data_dir = os.getenv("DATA_DIR")
output_dir = os.getenv("OUT_DIR")
output_dir = os.path.join(output_dir, model, run_id)
os.makedirs(output_dir, exist_ok=True)
df0 = get_db(os.path.join(data_dir, model, run_id))
df = df0.copy()
df = df[df['run_id'] == run_id]
len(df)

10

In [26]:
Du = 1
Dv = 18
A = 11
B = 55
df_filt = df[(df["Du"] == Du) & (df["Dv"] == Dv) & (df["A"] == A) & (df["B"] == B) & (df["random_seed"] == 1)]

In [27]:
row = df_filt.iloc[0]
ds = nc.Dataset(row["filename"])
data = data = ds.variables["data"][:]

make

u_min, u_std, u_max 10.634356 0.10000841 11.416812
v_min, v_std, v_max 4.634356 0.100008406 5.416812
u_min, u_std, u_max 9.952065 0.2904743 12.232405
v_min, v_std, v_max 4.664762 0.07873842 5.2839046
u_min, u_std, u_max 9.359256 0.44631267 12.7984495
v_min, v_std, v_max 4.5163746 0.119751304 5.4437118
u_min, u_std, u_max 8.439335 0.696322 13.6191
v_min, v_std, v_max 4.324046 0.1788906 5.686216
u_min, u_std, u_max 7.039512 1.1017263 14.954737
v_min, v_std, v_max 4.0638294 0.27258605 6.033211
u_min, u_std, u_max 5.093938 1.747302 17.010895
v_min, v_std, v_max 3.7036042 0.41598612 6.4143333
u_min, u_std, u_max 3.148084 2.726758 19.82764
v_min, v_std, v_max 3.2470284 0.6162809 6.6602798
u_min, u_std, u_max 1.852713 4.0749507 23.357878
v_min, v_std, v_max 2.7826347 0.8530987 7.012426
u_min, u_std, u_max 1.2960019 5.6864424 27.239985
v_min, v_std, v_max 2.3699431 1.0765922 7.1923156
u_min, u_std, u_max 0.9729499 7.3458424 30.936863
v_min, v_std, v_max 2.0563653 1.2376434 7.4166527
u_min, u_s

In [25]:
row = df_filt.iloc[0]
ds = nc.Dataset(row["filename"])
data = data = ds.variables["data"][:]
data = data[:, :13, :, :]
global_min = 0
global_max = 10

# Create a PdfPages object
pdf_filename = "blowup_1.pdf"

with PdfPages(pdf_filename) as pdf:
    for snapshot in range(13):
        # Create a figure for each snapshot
        fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(10, 10 * 2),
                                    gridspec_kw={'width_ratios': [1], 'height_ratios': [1] * 2, 'wspace': 0.1, 'hspace': 0.1})

        # plot only first functions
        # fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(10, 10),
                                    # gridspec_kw={'width_ratios': [1], 'height_ratios': [1], 'wspace': 0.1, 'hspace': 0.1})

        for coupled_idx in range(2):
            ax = axes[coupled_idx]

            # Extract the 2D matrix for the current member, snapshot, and coupled index
            matrix = data[0, snapshot, :, coupled_idx::2]

            # Plot the matrix
            im = ax.imshow(matrix, cmap='viridis', aspect='equal', vmin=global_min, vmax=global_max)
            ax.set_title(f'Snapshot {snapshot + 1}, Coupled Index {coupled_idx + 1}')
            ax.set_xlabel('y')
            ax.set_ylabel('x')
            # ax.set_xlim(0, y_size)
            # ax.set_ylim(x_size, 0)

        # Add a colorbar to the figure
        cbar = fig.colorbar(im, ax=axes, orientation='vertical', fraction=0.02, pad=0.04)
        cbar.set_label('Data Value')

        # Adjust layout to prevent overlap
        # plt.tight_layout(rect=[0, 0, 0.95, 1])  # Adjust rect to make room for colorbar

        # Save the figure to a file
        output_filename = 'blowup_1.pdf'
        pdf.savefig(bbox_inches='tight')
        plt.close(fig)  # Close the figure to free memory
